# Learning the Game of Life

This notebook contains an exploration on using convolutional neural networks to learn the simulation rules of Conway's Game of Life. Furthermore, I hope to exemplify in this notebook a situation where neural networks can be made to diverge from each other quickly during fine tuning.

## The Game of Life

Conway's Game of Life is commonly referred to as a zero player game set on an infinite field of cells. The term zero player refers to the fact that once the initial state of the infinite field of cells is set, no more human interaction is required. The game is played out by the following iteration rules. Applying them to the entire field is commonly called a step in the game of life.

1) Any live cell with fewer than two live neighbours dies, as if by underpopulation.
2) Any live cell with two or three live neighbours lives on to the next generation.
3) Any live cell with more than three live neighbours dies, as if by overpopulation.
4) Any dead cell with exactly three live neighbours becomes a live cell, as if by reproduction.

These rules are taken from the Wikipedia article [2]. When the Game of Life is played on a finite field, one common assumption made to simplify simulation is to assume that all cells beyond the finite field to be dead cells. This assumption was made by Kenyon and Springer during their analysis of learning the Game of Life using CNNs. However, this is found to be problematic since the behavior cannot be represented by the fundamental rules above. That is, the status of cells end up dependent on their distance from the edges. Hence, we propose a different scenario that involves periodic boundary conditions.

## Periodic Boundary Conditions

What we mean by periodic boundary conditions is that the edges and corners of the finite field effectively wrap around to the opposite edge/corner. For instance, the leftmost column acts as the column to the right of the rightmost column and vice versa. Likewise for the top and bottom rows. The top left corner is included in the step evaluation for the bottom right corner and vice versa. And similar behavior is seen for the other corners. This effectively changes the game of life to run on a torus surface.

In [1]:
import numpy as np
import time

In [2]:
initial_state = np.array(
[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0],
 [0, 0, 0, 0, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]])

In [3]:
"""
Parameters: A m by n binary matrix representing live (1) and dead (0) cells
            in the field.
Output: A m by n binary matrix representing the state of the field of cells
        after one step of the game of life. The step is evaluated on the
        input according to the periodic boundary conditions
"""
def step(game_state):
    m, n = np.shape(game_state)
    next_state = np.zeros((m, n), dtype=int)
    for i in range(m):
        for j in range(n):
            live_neighbors = 0
            
            live_neighbors += game_state[i-1,j-1]
            live_neighbors += game_state[i-1,j]
            live_neighbors += game_state[i-1,j+1 if j+1 < n else 0]
            
            live_neighbors += game_state[i,j-1]
            live_neighbors += game_state[i,j+1 if j+1 < n else 0]
            
            live_neighbors += game_state[i+1 if i+1 < m else 0, j-1]
            live_neighbors += game_state[i+1 if i+1 < m else 0, j]
            live_neighbors += game_state[i+1 if i+1 < m else 0, j+1 if j+1 < n else 0]
            
            if live_neighbors < 2: # dies by underpopulation (rule 1)
                next_state[i,j] = 0 
            elif live_neighbors > 3: # dies by overpopulation (rule 3)
                next_state[i,j] = 0 
            elif game_state[i,j] == 0 and live_neighbors == 3: # reproduction (rule 4)
                next_state[i,j] = 1
            elif game_state[i,j] == 1 and (live_neighbors == 2 or live_neighbors == 3): # (rule 2)
                next_state[i,j] = 1
    return next_state

def step_n(game_state, n=1):
    gs = game_state
    for i in range(n):
        gs = step(gs)
        
    return gs

In [4]:
gs = initial_state

In [5]:
from ipycanvas import MultiCanvas, hold_canvas

w=512
h=512

multi_canvas = MultiCanvas(2, width=w, height=h)
multi_canvas[0].fill_style = '#eceff4'
multi_canvas[0].fill_rect(0, 0, multi_canvas.width, multi_canvas.height)

In [6]:
def draw(x, canvas, color='#3b4252'):
    with hold_canvas(canvas):
        canvas.clear()
        canvas.fill_style = color
        
        m, n = np.shape(x)
        pixheight = canvas.height
        pixwidth = canvas.width
        m_pixels = pixheight / m
        n_pixels = pixwidth / n

        r = 0
        for row in x:
            c = 0
            for value in row:
                if value:
                    canvas.fill_rect(r * m_pixels, c * n_pixels, m_pixels, n_pixels)
                c += 1
            r += 1

In [7]:
multi_canvas

MultiCanvas(height=512, width=512)

In [48]:
def render_step(a):
    next_state = step(prev_state)
    draw(next_state, multi_canvas[1])

In [49]:
def test(a=None):
    print('Hello')

In [50]:
import ipywidgets as widgets

button = widgets.Button(
    description='Step',
    disabled=False,
    tooltip='Click me',
)

button.on_click(render_step)

button

Button(description='Step', style=ButtonStyle(), tooltip='Click me')

NameError: name 'prev_state' is not defined

# Bibliography
    
1. Springer, Jacob M., and Garrett T. Kenyon. "It's Hard for Neural Networks To Learn the Game of Life". CoRR, vol. abs/2009.01398, 2020, https://arxiv.org/abs/2009.01398.

2. https://en.wikipedia.org/wiki/Conway%27s_Game_of_Life